带有过滤器的交易策略
=============
***

1. 策略描述
----------
***

* 策略的信号来自于长期均线（120天）减去短期均线（10天）
* 备选股票的过去30日最大值必须大于15元

In [ ]:
%matplotlib inline
import datetime as dt

from AlgoTrading.Strategy.Strategy import Strategy
from AlgoTrading.Backtest import strategyRunner
from AlgoTrading.Backtest import DataSource
from PyFin.API import MA
from PyFin.API import MAX

* ``filter = MAX(30, 'high') > 15.0``：策略的过滤条件
* ``indicator = MA(10, 'close') - MA(120, 'close')``：策略的指标
* ``self.signal = indicator[filter]``：指标经过滤后产生信号

In [ ]:
class MovingAverageCrossStrategy(Strategy):
    def __init__(self):
        filtering = MAX(30, 'high') > 15.0
        indicator = MA(10, 'close') - MA(30, 'close')
        # 使用过滤器过滤出有效的指标
        self.signal = indicator[filtering]

    def handle_data(self):
        for s in self.universe:
            if self.signal[s] > 0. and self.secPos[s] == 0:
                self.order(s, 1, quantity=1000)
            elif self.signal[s] < 0. and self.secPos[s] != 0:
                self.order(s, -1, quantity=1000)

2. 策略运行设置
-----------------
***

需要设置策略运行的基本参数:

* ``universe``: *证券池*
* ``initialCapital``：*初始资金*
* ``startDate``：*回测开始日期*
* ``endDate``：*回测结束日期*
* ``dataSource``：*数据源*
* ``freq``：回测时间频率（分钟），仅当使用``DataSource.DXDataCenter``时有效。

In [ ]:
def run_example():
    universe = ['000001.XSHE', '000002.XSHE', '000004.XSHE', '000005.XSHE', '000006.XSHE', '000007.XSHE', '000008.XSHE']
    initialCapital = 100000.0
    startDate = dt.datetime(2006, 10, 1)
    endDate = dt.datetime(2015, 10, 1)

    strategyRunner(userStrategy=MovingAverageCrossStrategy,
                   initialCapital=initialCapital,
                   symbolList=universe,
                   startDate=startDate,
                   endDate=endDate,
                   dataSource=DataSource.DataYes,
                   logLevel='critical',
                   benchmark='000300.zicn',
                   freq=1,
                   saveFile=True,
                   plot=True)

3. 策略运行并输出
-----------------
***

In [ ]:
run_example()